In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import json

In [ ]:
with open('kaggle.json') as f:
    key = json.load(f)["Roboflow"]
rf = Roboflow(api_key=key)

project = rf.workspace("dmay").project("cctv_car_bike_detection-jzjis")
dataset = project.version(3).download("yolov8", "./datasets/car_bike")

project = rf.workspace("ujjawal").project("seat-belt-2-77oye")
dataset = project.version(5).download("yolov8", "./datasets/seatbelt")

project = rf.workspace("kalay-highschool").project("yahli-helmet-no-helmet")
dataset = project.version(1).download("yolov8", "./datasets/helmet")

project = rf.workspace("muhammad-rizki").project("plat-nomor-detection")
dataset = project.version(2).download("yolov8", "./datasets/plat")

In [ ]:
PATH = "/mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/"
model = YOLO('./datasets/yolov8s.pt')
results = model.train(data=PATH+'car_bike/data.yaml', epochs=100, batch=32, imgsz=960, device=0)

In [ ]:
PATH = "/mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/"
model = YOLO('./datasets/yolov8s.pt')
results = model.train(data=PATH+'helmet/32.yaml', epochs=100, batch=32, imgsz=960, device=0)

In [ ]:
PATH = "/mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/"
model = YOLO('./datasets/yolov8s.pt')
results = model.train(data=PATH+'plat/32.yaml', epochs=100, batch=32, imgsz=960, device=0)

In [ ]:
PATH = "/mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/"
model = YOLO('./datasets/yolov8s.pt')
results = model.train(data=PATH+'seatbelt/data.yaml', epochs=100, batch=32, imgsz=960, device=0)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2
from keras.optimizers import SGD
import opendatasets

project = 'https://www.kaggle.com/sachinpatel21/az-handwritten-alphabets-in-csv-format'
opendatasets.download(project, "./datasets")

In [ ]:
def load_az_dataset(datasetPath):
	data = []
	labels = []

	for row in open(datasetPath):
		row = row.split(",")
		label = int(row[0])
		image = np.array([int(x) for x in row[1:]], dtype="uint8")
		image = image.reshape((28, 28))
		data.append(image)
		labels.append(label)

	data = np.array(data, dtype="float32")
	labels = np.array(labels, dtype="int")
	return (data, labels)

def load_mnist_dataset():
	((trainData, trainLabels), (testData, testLabels)) = tf.keras.datasets.mnist.load_data()
	data = np.vstack([trainData, testData])
	labels = np.hstack([trainLabels, testLabels])
	return (data, labels)

In [ ]:
(azData, azLabels) = load_az_dataset("./datasets/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")
(digitsData, digitsLabels) = load_mnist_dataset()

azLabels += 10
data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])
data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")
data = np.expand_dims(data, axis=-1)
data /= 255.0

le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis=0)
classTotals = labels.sum(axis=0)
classWeight = {}
for i in range(0, len(classTotals)):
	classWeight[i] = classTotals.max() / classTotals[i]
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

aug = tf.keras.preprocessing.image.ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	fill_mode="nearest")

In [ ]:
def residual_module(data, K, stride, red=False, reg=0.0001):
    shortcut = data
    bn1 = tf.keras.layers.BatchNormalization(axis=-1)(data)
    act1 = tf.keras.layers.Activation("relu")(bn1)
    conv1 = tf.keras.layers.Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(reg))(act1)

    bn2 = tf.keras.layers.BatchNormalization(axis=-1)(conv1)
    act2 = tf.keras.layers.Activation("relu")(bn2)
    conv2 = tf.keras.layers.Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(reg))(act2)

    bn3 = tf.keras.layers.BatchNormalization(axis=-1)(conv2)
    act3 = tf.keras.layers.Activation("relu")(bn3)
    conv3 = tf.keras.layers.Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(reg))(act3)

    if red:
        shortcut = tf.keras.layers.Conv2D(K, (1, 1), strides=stride, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(reg))(act1)
    x = tf.keras.layers.add([conv3, shortcut])
    return x

stages = [3, 3, 3]
filters = [64, 64, 128, 256]

inputs = tf.keras.layers.Input(shape=(32, 32, 1))
x = tf.keras.layers.BatchNormalization(axis=-1)(inputs)
x = tf.keras.layers.Conv2D(64, (3, 3), use_bias=False, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)

for i in range(0, len(stages)):
    stride = (1, 1) if i == 0 else (2, 2)
    x = residual_module(x, filters[i + 1], stride, red=True)

    for _ in range(0, stages[i] - 1):
        x = residual_module(x, filters[i + 1], (1, 1))

x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.AveragePooling2D((8, 8))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(len(le.classes_), kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
x = tf.keras.layers.Activation("softmax")(x)

model = tf.keras.models.Model(inputs, x)

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=SGD(learning_rate=0.1, weight_decay=0.002),
    metrics=["accuracy"])

H = model.fit(
	aug.flow(trainX, trainY, batch_size=128),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // 128,
	epochs=10,
	class_weight=classWeight,
	verbose=1)

model.save("./datasets/ocr.h5", save_format="h5")

In [9]:
from ultralytics import YOLO


car_bike = YOLO("./runs/detect/train/weights/best.pt")
helmet = YOLO("./runs/detect/train2/weights/best.pt")
plat = YOLO("./runs/detect/train3/weights/best.pt")
seatbelt = YOLO("./runs/detect/train4/weights/best.pt")
ocr = tf.keras.models.load_model("./datasets/ocr.h5")

metrics1 = car_bike.val()
metrics2 = helmet.val()
metrics3 = plat.val()
metrics4 = seatbelt.val()
metrics5 = ocr.evaluate(testX, testY, batch_size=128)

Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/car_bike/valid/labels.cache... 159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 159/159 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 30, len(boxes) = 1854. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:22<00:00,  2.26s/it]


                   all        159       1854      0.848       0.84      0.889       0.58
                  bike        159        373      0.851      0.831      0.878      0.549
                   car        159       1481      0.845      0.849        0.9      0.612
Speed: 2.4ms preprocess, 118.3ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


...

                   all         49        145      0.772      0.589      0.672      0.384
           With Helmet         49         37      0.798      0.427      0.578      0.333
        Without Helmet         49        108      0.747       0.75      0.765      0.434
Speed: 3.3ms preprocess, 183.1ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val2
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/plat/valid/labels.cache... 181 images, 0 backgrounds, 0 corrupt: 100%|██████████| 181/181 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:30<00:00,  2.52s/it]


                   all        181        224      0.966      0.946      0.974        0.8
Speed: 1.2ms preprocess, 154.1ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val3
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/c/Users/sahri/OneDrive/Documents/VSCode/Bangkit/Machine_learning/datasets/seatbelt/valid/labels.cache... 148 images, 0 backgrounds, 0 corrupt: 100%|██████████| 148/148 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:23<00:00,  2.31s/it]


                   all        148        151      0.997          1      0.995      0.722
              seatbelt        148         74      0.998          1      0.995      0.696
           no-seatbelt        148         77      0.996          1      0.995      0.748
Speed: 3.1ms preprocess, 143.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val4
692/692 [==============================] - 9s 12ms/step - loss: 0.3299 - accuracy: 0.9233


In [11]:
!cp "./runs/detect/train/weights/best.pt" "./models/vehicle.pt"
!cp "./runs/detect/train2/weights/best.pt" "./models/helmet.pt"
!cp "./runs/detect/train3/weights/best.pt" "./models/plate.pt"
!cp "./runs/detect/train4/weights/best.pt" "./models/seatbelt.pt"
!cp "./runs/detect/train/confusion_matrix_normalized.png" "./models/confusion_matrix_normalized_vehicle.png"
!cp "./runs/detect/train2/confusion_matrix_normalized.png" "./models/confusion_matrix_normalized_helmet.png"
!cp "./runs/detect/train3/confusion_matrix_normalized.png" "./models/confusion_matrix_normalized_plate.png"
!cp "./runs/detect/train4/confusion_matrix_normalized.png" "./models/confusion_matrix_normalized_seatbelt.png"
!cp "./datasets/ocr.h5" "./models/ocr.h5"